In [1]:
# ============================================================
# EVE Tycoon – Market Debug Notebook
# Skills | Blueprints | Weapons
# ============================================================

import requests
import pandas as pd
import time

BASE_URL = "https://evetycoon.com/api"

# ------------------------------------------------------------
# 1️⃣ Hilfsfunktionen
# ------------------------------------------------------------

def get_json(url):
    """GET-Request mit Wiederholversuchen & Textausgabe"""
    for attempt in range(3):
        try:
            r = requests.get(url, timeout=20)
            if r.status_code == 200:
                return r.json()
            else:
                print(f"⚠️  Fehler {r.status_code} bei {url}")
        except Exception as e:
            print(f"⚠️  Netzwerkfehler bei {url}: {e}")
        print("   ↪️  Versuch erneut in 2 Sekunden...")
        time.sleep(2)
    print(f"❌ Keine Antwort von {url}")
    return None


def get_market_groups():
    """Alle Market-Gruppen abrufen"""
    print("🔹 Lade Market-Groups...")
    data = get_json(f"{BASE_URL}/v1/market/groups")
    if not data:
        raise RuntimeError("❌ Keine Market-Groups erhalten.")
    print(f"✅ {len(data)} Market-Groups geladen.\n")
    return data


def get_items_in_group(group_id, group_name):
    """Alle Items innerhalb einer Market-Gruppe abrufen"""
    print(f"   → Lade Items aus Gruppe: {group_name}")
    data = get_json(f"{BASE_URL}/v1/market/groups/{group_id}/types")
    if data:
        print(f"     ✅ {len(data)} Items gefunden.")
    else:
        print("     ⚠️  Keine Items gefunden.")
    return data or []


def get_orders_for_item(type_id, type_name):
    """Alle Orders für ein Item abrufen"""
    url = f"{BASE_URL}/v1/market/orders/{type_id}"
    data = get_json(url)
    if not data or "orders" not in data:
        print(f"     ⚠️  Keine Orders für {type_name}")
        return None
    return data


# ------------------------------------------------------------
# 2️⃣ Relevante Gruppen finden
# ------------------------------------------------------------

groups = get_market_groups()

target_groups = [
    g for g in groups
    if any(keyword in g["marketGroupName"].lower() for keyword in ["skill", "blueprint", "weapon"])
]

print(f"🎯 Gefundene relevante Gruppen: {len(target_groups)}")
for g in target_groups:
    print(f"   - {g['marketGroupName']}")
print("")

# ------------------------------------------------------------
# 3️⃣ Alle Items aus diesen Gruppen laden
# ------------------------------------------------------------

all_items = []
for g in target_groups:
    items = get_items_in_group(g["marketGroupID"], g["marketGroupName"])
    all_items.extend(items)
    time.sleep(0.2)

print(f"\n📦 Gesamtanzahl Items: {len(all_items)}\n")

# ------------------------------------------------------------
# 4️⃣ Daten sammeln: günstigste & teuerste Verkaufsorders
# ------------------------------------------------------------

rows = []
print("🚀 Beginne mit Abruf der Orders...\n")

for i, item in enumerate(all_items, 1):
    type_id = item["typeID"]
    type_name = item["typeName"]
    print(f"[{i}/{len(all_items)}] → Lade Orders für {type_name}")

    orders_data = get_orders_for_item(type_id, type_name)
    if not orders_data:
        continue

    # Nur Sell Orders mit Bestand
    sell_orders = [o for o in orders_data["orders"] if not o["isBuyOrder"] and o["volumeRemain"] > 0]
    if len(sell_orders) == 0:
        print("   ⚠️  Keine aktiven Sell Orders gefunden.")
        continue

    sell_orders_sorted = sorted(sell_orders, key=lambda x: x["price"])
    cheapest5 = sell_orders_sorted[:5]
    expensive5 = sell_orders_sorted[-5:]

    stations = orders_data.get("stationNames", {})
    systems = orders_data.get("systems", {})
    structures = orders_data.get("structureNames", {})

    for tag, order_list in [("Cheapest", cheapest5), ("MostExpensive", expensive5)]:
        for o in order_list:
            loc_id = str(o["locationId"])
            sys_id = str(o["systemId"])

            region_id = o["regionId"]
            system_name = systems.get(sys_id, {}).get("solarSystemName", sys_id)
            region_name = systems.get(sys_id, {}).get("regionName", region_id)
            station_name = stations.get(loc_id, "")
            structure_name = structures.get(loc_id, "")

            rows.append({
                "typeID": type_id,
                "typeName": type_name,
                "orderType": tag,
                "price": o["price"],
                "volumeRemain": o["volumeRemain"],
                "regionName": region_name,
                "systemName": system_name,
                "stationName": station_name,
                "structureName": structure_name
            })

    time.sleep(0.2)

# ------------------------------------------------------------
# 5️⃣ Debug-Auswertung
# ------------------------------------------------------------

print("\n=== Debug Report ===")
print(f"Gefundene Orders insgesamt: {len(rows)}")

if len(rows) == 0:
    print("⚠️  Es wurden keine Orders gefunden. Teste manuell mit PLEX.")
    test_url = f"{BASE_URL}/v1/market/orders/44992"
    test_data = get_json(test_url)
    print(f"   Anzahl Orders für PLEX: {len(test_data.get('orders', [])) if test_data else 0}")
else:
    print("✅ Daten wurden gesammelt – erstelle DataFrame...")

# ------------------------------------------------------------
# 6️⃣ Anzeige im Notebook
# ------------------------------------------------------------

if len(rows) > 0:
    df = pd.DataFrame(rows)
    df_sorted = df.sort_values(by=["typeName", "orderType", "price"]).reset_index(drop=True)

    pd.set_option("display.max_rows", 30)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 200)

    print(f"\n✅ Fertig! {len(df_sorted)} Orders geladen.")
    print("\n📈 Beispielauszug:\n")
    display(df_sorted.head(20))
else:
    print("❌ Keine Daten zum Anzeigen.")


🔹 Lade Market-Groups...
✅ 2038 Market-Groups geladen.

🎯 Gefundene relevante Gruppen: 51
   - Blueprints & Reactions
   - Weapon Upgrades
   - Skills
   - Weapon Upgrades
   - Weapon Batteries
   - Skill Hardwiring
   - Weapon Disruptors
   - Superweapons
   - Superweapons
   - Energy Weapon Rigs
   - Hybrid Weapon Rigs
   - Projectile Weapon Rigs
   - Energy Weapon Rigs
   - Hybrid Weapon Rigs
   - Projectile Weapon Rigs
   - Small Energy Weapon Rigs
   - Medium Energy Weapon Rigs
   - Large Energy Weapon Rigs
   - Small Hybrid Weapon Rigs
   - Medium Hybrid Weapon Rigs
   - Large Hybrid Weapon Rigs
   - Small Projectile Weapon Rigs
   - Medium Projectile Weapon Rigs
   - Large Projectile Weapon Rigs
   - Small Energy Weapon Rigs
   - Medium Energy Weapon Rigs
   - Large Energy Weapon Rigs
   - Small Hybrid Weapon Rigs
   - Medium Hybrid Weapon Rigs
   - Large Hybrid Weapon Rigs
   - Small Projectile Weapon Rigs
   - Medium Projectile Weapon Rigs
   - Large Projectile Weapon Rigs
   -

,typeID,typeName,orderType,price,volumeRemain,regionName,systemName,stationName,structureName
0,77402,'Atgeir' Explosive Disruptive Lance Blueprint,Cheapest,500000000.0,16,10000028,Hrober,Hrober VI - Republic Fleet Testing Facilities,
1,77402,'Atgeir' Explosive Disruptive Lance Blueprint,Cheapest,500000000.0,16,10000028,Hrober,Hrober VII - Moon 5 - Republic Fleet Testing F...,
2,77402,'Atgeir' Explosive Disruptive Lance Blueprint,Cheapest,500000000.0,16,10000028,Sakulda,Sakulda IX - Moon 2 - Republic Fleet Testing F...,
3,77402,'Atgeir' Explosive Disruptive Lance Blueprint,Cheapest,500000000.0,16,10000028,Hedaleolfarber,Hedaleolfarber IX - Moon 23 - Republic Fleet T...,
4,77402,'Atgeir' Explosive Disruptive Lance Blueprint,Cheapest,500000000.0,16,10000028,Half,Half X - Moon 10 - Republic Fleet Assembly Plant,
5,77402,'Atgeir' Explosive Disruptive Lance Blueprint,MostExpensive,500000000.0,33,10000001,Uzistoon,Uzistoon VII - Moon 2 - Thukker Mix Factory,
6,77402,'Atgeir' Explosive Disruptive Lance Blueprint,MostExpensive,500000000.0,33,10000001,Nakah,Nakah I - Moon 1 - Thukker Mix Factory,
7,77402,'Atgeir' Explosive Disruptive Lance Blueprint,MostExpensive,697000000.0,1,10000032,Dodixie,Dodixie IX - Moon 20 - Federation Navy Assembl...,
8,77402,'Atgeir' Explosive Disruptive Lance Blueprint,MostExpensive,699800000.0,1,10000043,Amarr,Amarr VIII (Oris) - Emperor Family Academy,
9,77402,'Atgeir' Explosive Disruptive Lance Blueprint,MostExpensive,699900000.0,2,10000002,Jita,Jita IV - Moon 4 - Caldari Navy Assembly Plant,


In [3]:
# ============================================================
# SELL→SELL Arbitrage über alle Sell-Orders je Station (bereinigt)
# - pro Station nur der niedrigste Verkaufspreis zählt
# - Verkauf = Station mit größtem Spread relativ zum globalen Einkauf
# ============================================================

import time
import pandas as pd
import datetime

# --- Sicherheits-Checks / Hilfen ---
if "all_items" not in locals():
    raise RuntimeError("⚠️ 'all_items' fehlt. Bitte zuerst die Zellen ausführen, die Market-Gruppen/Items laden.")

if "get_orders_for_item" not in locals():
    raise RuntimeError("⚠️ 'get_orders_for_item' fehlt. Bitte die API-Hilfsfunktionen-Zelle ausführen.")

def _label_row(system_name, region_name, station_name, structure_name):
    place = station_name if station_name else structure_name
    return f"{place} ({system_name}, {region_name})"

print("\n📦 Lade ALLE Sell-Orders für die ausgewählten Items (Station-basierte Aggregation)...")

all_sell_rows = []

for idx, item in enumerate(all_items, 1):
    type_id = item["typeID"]
    type_name = item["typeName"]
    print(f"[{idx}/{len(all_items)}] → Orders für: {type_name}")

    data = get_orders_for_item(type_id, type_name)
    if not data or "orders" not in data:
        continue

    stations = data.get("stationNames", {})
    structures = data.get("structureNames", {})
    systems = data.get("systems", {})

    # ALLE Sell-Orders (nicht nur Top 5/Top N)
    sells = [o for o in data["orders"] if not o.get("isBuyOrder", False) and o.get("volumeRemain", 0) > 0]
    if not sells:
        continue

    for o in sells:
        loc_id = str(o["locationId"])
        sys_id = str(o["systemId"])
        region_name = systems.get(sys_id, {}).get("regionName", o.get("regionId"))
        system_name = systems.get(sys_id, {}).get("solarSystemName", sys_id)
        station_name = stations.get(loc_id, "")
        structure_name = structures.get(loc_id, "")

        all_sell_rows.append({
            "typeID": type_id,
            "typeName": type_name,
            "locationId": loc_id,                 # 🔑 stabile Station/Struktur-ID
            "price": float(o["price"]),
            "volumeRemain": int(o.get("volumeRemain", 0)),
            "regionName": region_name,
            "systemName": system_name,
            "stationName": station_name,
            "structureName": structure_name,
            "Ort": _label_row(system_name, region_name, station_name, structure_name)
        })
    time.sleep(0.15)  # etwas sanfter fürs Rate-Limit

df_sells_all = pd.DataFrame(all_sell_rows)
if df_sells_all.empty:
    raise RuntimeError("❌ Keine Sell-Orders gefunden. Bitte vorherige Schritte/Filter prüfen.")

print(f"✅ Sell-Orders geladen: {len(df_sells_all):,}")

# ------------------------------------------------------------
# Gruppierung: pro Item & Station der MINDEST-PREIS (Station nur 1x)
# ------------------------------------------------------------
# Wir gruppieren über (typeName, locationId) und nehmen:
# - min_price: niedrigster Verkaufspreis an dieser Station
# - total_vol: Summe des noch verfügbaren Volumens an dieser Station (optional nützlich)
station_min_prices = (
    df_sells_all
    .groupby(["typeName", "locationId"], as_index=False)
    .agg(
        min_price=("price", "min"),
        total_vol=("volumeRemain", "sum"),
        Ort=("Ort", "first"),                     # repräsentatives Label
        regionName=("regionName", "first"),
        systemName=("systemName", "first"),
        stationName=("stationName", "first"),
        structureName=("structureName", "first"),
    )
)

# ------------------------------------------------------------
# Für jedes Item:
# - Einkauf: globaler Mindestpreis (über alle Stationen)
# - Verkauf: Station mit MAX(min_price) > buy_price (größter Spread), Station ≠ Einkaufs-Station
# ------------------------------------------------------------
records = []

for item_name, g in station_min_prices.groupby("typeName"):
    if len(g) < 2:
        continue

    # Einkauf
    buy_idx = g["min_price"].idxmin()
    buy_row = g.loc[buy_idx]
    buy_price = buy_row["min_price"]
    buy_loc_id = buy_row["locationId"]
    buy_loc_lbl = buy_row["Ort"]

    # Kandidaten für Verkauf: alle Stationen mit höherem Mindestpreis als Einkauf
    candidates = g[g["min_price"] > buy_price].copy()
    if candidates.empty:
        continue

    # Spread berechnen
    candidates["spread_abs"] = candidates["min_price"] - buy_price
    candidates["spread_pct"] = candidates["spread_abs"] / buy_price * 100

    # Sicherstellen, dass wir NICHT die gleiche Station wählen
    candidates = candidates[candidates["locationId"] != buy_loc_id]
    if candidates.empty:
        continue

    # Beste Verkaufsstation = größte spread_pct
    sell_row = candidates.sort_values("spread_pct", ascending=False).iloc[0]
    sell_price = sell_row["min_price"]
    sell_loc_lbl = sell_row["Ort"]

    records.append({
        "Item": item_name,
        "Einkaufspreis [ISK]": buy_price,
        "Einkaufs-Ort": buy_loc_lbl,
        "Verkaufspreis [ISK] (niedrigste Sell an Zielstation)": sell_price,
        "Verkaufs-Ort": sell_loc_lbl,
        "Spread [ISK]": sell_price - buy_price,
        "Spread [%]": (sell_price - buy_price) / buy_price * 100,
        "Volumen (Einkaufs-Station)": int(buy_row["total_vol"]),
        "Volumen (Verkaufs-Station)": int(sell_row["total_vol"]),
    })

spread_sell_to_sell = (
    pd.DataFrame(records)
    .sort_values("Spread [%]", ascending=False)
    .reset_index(drop=True)
)

print(f"✅ Tabelle erstellt: {len(spread_sell_to_sell):,} Items")

# Anzeige hübsch formatieren
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 320)

print("\n📈 Top 500 Sell→Sell-Spreads (Station-aggregiert, je Station min Preis):\n")
display(spread_sell_to_sell.head(500))

# ------------------------------------------------------------
# 💾 Export mit Datum
# ------------------------------------------------------------
today = datetime.date.today().strftime("%Y-%m-%d")
filename = f"Eve_Spread_500_{today}_sell_to_sell_clean.csv"
spread_sell_to_sell.head(500).to_csv(filename, index=False)
print(f"💾  Datei '{filename}' gespeichert.")



📦 Lade ALLE Sell-Orders für die ausgewählten Items (Station-basierte Aggregation)...
[1/305] → Orders für: Gyrostabilizer I Blueprint
[2/305] → Orders für: Magnetic Field Stabilizer I Blueprint
[3/305] → Orders für: Heat Sink I Blueprint
[4/305] → Orders für: Tracking Enhancer I Blueprint
[5/305] → Orders für: Remote Tracking Computer I Blueprint
[6/305] → Orders für: Tracking Computer I Blueprint
[7/305] → Orders für: Ballistic Control System I Blueprint
[8/305] → Orders für: Siege Module I Blueprint
[9/305] → Orders für: Triage Module I Blueprint
[10/305] → Orders für: Capital Industrial Core I Blueprint
[11/305] → Orders für: Bastion Module I Blueprint
[12/305] → Orders für: Missile Guidance Enhancer I Blueprint
[13/305] → Orders für: Missile Guidance Computer I Blueprint
[14/305] → Orders für: Entropic Radiation Sink I Blueprint
[15/305] → Orders für: ML-EKP 'Polybolos' Ballistic Control System Blueprint
[16/305] → Orders für: Veles Entropic Radiation Sink Blueprint
[17/305] → Ord

,Item,Einkaufspreis [ISK],Einkaufs-Ort,Verkaufspreis [ISK] (niedrigste Sell an Zielstation),Verkaufs-Ort,Spread [ISK],Spread [%],Volumen (Einkaufs-Station),Volumen (Verkaufs-Station)
0,DDO Scoped Tracking Disruptor I,12.12,"Cleyd V - Carthum Conglomerate Factory (Cleyd,...","1,710,000.00",PR-8CA III - Blood Raiders Logistic Support (P...,"1,709,987.88","14,108,810.89",1,94
1,Balmer Series Compact Tracking Disruptor I,144.10,"Cleyd V - Carthum Conglomerate Factory (Cleyd,...","2,499,000.00",1DH-SX III - Moon 1 - Blood Raiders Logistic S...,"2,498,855.90","1,734,112.35",1,12
2,C-IR Compact Guidance Disruptor,100.10,Clellinon VI - Moon 11 - Center for Advanced S...,"990,900.00",319-3D IX - Moon 22 - Blood Raiders Logistic S...,"990,799.90","989,810.09",3,38
3,Highstroke Scoped Guidance Disruptor,"3,500.00",Rens VI - Moon 8 - Brutor Tribe Treasury (Rens...,"22,230,000.00","Jita VI - Paragon Fulfillment Center (Jita, 10...","22,226,500.00","635,042.86",75,1
4,Tracking Disruptor I,"29,830.00",Thera XII - The Sanctuary Surveillance Observa...,"25,000,000.00",Berta VI - Moon 19 - Ammatar Consulate Bureau ...,"24,970,170.00","83,708.25",9,1
5,A-211 Enduring Guidance Disruptor,"3,005.00",Korama II - Moon 5 - Republic Security Service...,"1,000,000.00",Boystin V - Moon 6 - Federal Intelligence Offi...,"996,995.00","33,177.87",4,20
6,Large Energy Ambit Extension II,"900,000.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,"149,900,000.00",Apanake VIII - Moon 8 - Sisters of EVE Bureau ...,"149,000,000.00","16,555.56",8,1
7,Small Energy Discharge Elutriation I,"30,000.00",Pucherie VI - Moon 11 - Federation Navy Testin...,"3,600,000.00",Zarzakh - Deathless Custodians - The Fulcrum (...,"3,570,000.00","11,900.00",13,20
8,Small Hybrid Burst Aerator I,"50,000.00",Aimoguier V - Material Acquisition Mining Outp...,"5,950,000.00",Trossere VII - Moon 3 - University of Caille (...,"5,900,000.00","11,800.00",3,1
9,Medium Algid Energy Administrations Unit II,"175,600.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,"18,000,000.00",Rens VI - Moon 8 - Brutor Tribe Treasury (Rens...,"17,824,400.00","10,150.57",52,3


💾  Datei 'Eve_Spread_500_2025-10-14_sell_to_sell_clean.csv' gespeichert.
